In [1]:
import cartopy
import cartopy.crs as ccrs
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import bathyutils

First we set the extent of our map in degrees

In [3]:
S_lim = 57
N_lim = 59
W_lim = 17
E_lim = 23

Next we subset bathymetric data from either [emodnet](https://portal.emodnet-bathymetry.eu/) (hi-res, Europe only) or [GEBCO ](https://download.gebco.net/)(global). To subset the data, we can use the functions `bathyutils.gebco_subset` or `bathyutils.emod_subset`. Both functions require the path to the bathymetry datasets, and the extent of the area to be mapped

In [4]:
bathyutils.emod_subset?

Signature: bathyutils.emod_subset(extent, path_to_emodnet, buffer=0.2)
Docstring:
Selects the EMODnet bathymetry tiles that cover the user defined area and
combines them for a seamless bathymetry. Returns lon, lat and bathy.
:param extent: list with four items which are extent of desired geotiff [South, North, West, East]
e.g. [49. 50.5, -5, 2] (if using gebco or emodnet)
:param path_to_emodnet: string or path to the folder containing EMODnet .nc files, specified by user
:param buffer: size in degrees of extra area around specified coordinates to return. Detaults to 0.2.
:return: xarray object of subset bathymetry
File:      ~/Documents/data-flow/bathy_maps/bathyutils.py
Type:      function


In [5]:
emodnet_path = "/home/callum/Documents/datasets/bathy/emodnet"
extent = (S_lim, N_lim, W_lim, E_lim)
ds_bathy = bathyutils.emod_subset(extent, emodnet_path)

Found tiles [PosixPath('/home/callum/Documents/datasets/bathy/emodnet/C6_2020.nc'), PosixPath('/home/callum/Documents/datasets/bathy/emodnet/D7_2020.nc'), PosixPath('/home/callum/Documents/datasets/bathy/emodnet/C7_2020.nc'), PosixPath('/home/callum/Documents/datasets/bathy/emodnet/C5_2020.nc'), PosixPath('/home/callum/Documents/datasets/bathy/emodnet/D5_2020.nc'), PosixPath('/home/callum/Documents/datasets/bathy/emodnet/D6_2020.nc')]
Found 1 tiles with relevant data: [PosixPath('/home/callum/Documents/datasets/bathy/emodnet/D6_2020.nc')]


In [ ]:
coord = cartopy.crs.AzimuthalEquidistant(central_longitude=np.mean((W_lim, E_lim)),
                                  central_latitude=np.mean((S_lim, N_lim)))
pc = cartopy.crs.PlateCarree()
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection=coord)

bathy_c = ax.contourf(ds_bathy.lon, ds_bathy.lat, ds_bathy.elevation, transform=pc, cmap="Blues_r")
ax.set_extent((W_lim, E_lim, S_lim, N_lim), crs=pc)

feature = cartopy.feature.NaturalEarthFeature(name='land', category='physical',
                                       scale='10m', edgecolor='black', facecolor='lightgreen')
ax.add_feature(feature)
gl = ax.gridlines(draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = None
gl.right_labels = None
fig.colorbar(bathy_c,ax=ax,orientation='horizontal',label='Bathymetry (m)',shrink=0.7,pad=0.1,aspect=40)
